In [1]:
import os
import random
import math

import numpy as np
import pandas as pd
from PIL import Image, ImageDraw


import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler

import torchvision
from torchvision import datasets, models, transforms
from torchvision.models.detection.retinanet import RetinaNet
from torchvision.models.detection.faster_rcnn import FasterRCNN
import  torchvision.transforms.functional as F
from torchvision.models.detection.anchor_utils import AnchorGenerator


from typing import Any, Callable, Dict, List, Optional, Tuple, Union
import xml.etree.ElementTree as ET
import collections
from torchvision.datasets.voc import VisionDataset

from functions import *
from functions_torch import *

In [2]:
params = {}
params['target_size']=(2000,1500)

DSRoots = ['../../ladd-and-weights/dataset/'+d for d in
    ['LADD/summer_nnovgorod_2021', 'LADD/spring_korolev_2019', 'LADD/summer_moscow_2019', 'LADD/summer_tambov_2019', 'LADD/winter_moscow_2018','full_train_ds' ]]

In [3]:
# For GPU evaluation/test
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False, num_classes=2, pretrained_backbone=True, 
                                                           min_size=params['target_size'][0], max_size = params['target_size'][1],
                                                           trainable_backbone_layers = 5)
model.load_state_dict(torch.load('../../ladd-and-weights/weights/torch/experimental/resnet50_FRCNN_LADD_epoch_9.pth'), strict=True)
# model.load_state_dict(torch.load('/app/host/lacmus/weights/resnet50_FRCNN_LADD_epoch_9.pth'), strict=True)
model = model.to(torch.device('cuda'))

In [4]:
# For CPU evaluation/test
# model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False, num_classes=2, pretrained_backbone=True, 
#                                                            min_size=params['target_size'][0], max_size = params['target_size'][1],
#                                                            trainable_backbone_layers = 5)
# model.load_state_dict(torch.load('resnet50_FRCNN_LADD_epoch_9.pth', map_location=torch.device('cpu')), strict=True)
# model = model.to(torch.device('cpu'))
# model.eval()


In [5]:
class LADDDataSET(torchvision.datasets.VisionDataset):
    def __init__(
            self,
            root: str,
            image_set: str,
            transforms: Optional[Callable] = None):     
        super(LADDDataSET, self).__init__(root, transforms=transforms)
        self.image_set = image_set

        voc_root = root
        image_dir = os.path.join(voc_root, 'JPEGImages')
        annotation_dir = os.path.join(voc_root, 'Annotations')

        if not os.path.isdir(voc_root):
            raise RuntimeError('Dataset not found or corrupted.')

        splits_dir = os.path.join(voc_root, 'ImageSets/Main')
        split_f = os.path.join(splits_dir, image_set.rstrip('\n') + '.txt')

        with open(os.path.join(split_f), "r") as f:
            file_names = [x.strip() for x in f.readlines()]

        self.images = [os.path.join(image_dir, x + ".jpg") for x in file_names]
        self.annotations = [os.path.join(annotation_dir, x + ".xml") for x in file_names]
        assert (len(self.images) == len(self.annotations))
        
    def __getitem__(self, index: int) -> Tuple[Any, Any]:
        """
        Args:
            index (int): Index

        Returns:
            tuple: (image, target) where target is a dictionary of the XML tree.
        """
        img = Image.open(self.images[index]).convert('RGB')
        description = LADDDataSET.parse_voc_xml(
            ET.parse(self.annotations[index]).getroot())

        # get bounding box coordinates 
        num_objs = len(description['annotation']['object'])
        boxes = []
        for l in description['annotation']['object']:
            bb = l['bndbox']
            boxes.append([int(bb['xmin']), int(bb['ymin']), int(bb['xmax']), int(bb['ymax'])])

        target = {}
        target["boxes"] = torch.as_tensor(boxes, dtype=torch.float32)                
        target["labels"] = labels = torch.ones((num_objs,), dtype=torch.int64)  # there is only one class   
        
        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target
    

    def __len__(self) -> int:
        return len(self.images)

    @staticmethod
    def parse_voc_xml(node: ET.Element) -> Dict[str, Any]:
        voc_dict: Dict[str, Any] = {}
        children = list(node)
        if children:
            def_dic: Dict[str, Any] = collections.defaultdict(list)
            for dc in map(LADDDataSET.parse_voc_xml, children):
                for ind, v in dc.items():
                    def_dic[ind].append(v)
            if node.tag == 'annotation':
                def_dic['object'] = [def_dic['object']]
            voc_dict = {
                node.tag:
                    {ind: v[0] if len(v) == 1 else v
                     for ind, v in def_dic.items()}
            }
        if node.text:
            text = node.text.strip()
            if not children:
                voc_dict[node.tag] = text
        return voc_dict

In [6]:
def convert_yolo_to_pixels(size,yolo_string):
    s=[float(s) for s in  yolo_string.strip().split() ]
    center_x=int(s[1]*size[0])
    center_y=int(s[2]*size[1])
    w=int(s[3]*size[0])
    h=int(s[4]*size[1])
    x1=int(center_x-w/2)
    x2=int(center_x+w/2)
    y1=int(center_y-h/2)
    y2=int(center_y+h/2)
    return (x1,y1),(x2,y2),float(s[5])

In [7]:
for ds in DSRoots:
    dataset = LADDDataSET(ds,'test',get_transform(train=False,target_size=params['target_size'])) 
    data_loader = torch.utils.data.DataLoader(
        dataset, batch_size=1, shuffle=False, num_workers=1,collate_fn=collate_fn)
    inference_res = evaluate(model,data_loader, device='cuda')
    print("Inference for %s, computing mAp : "%ds)
    print(evaluate_res(inference_res, iou_threshold = 0.5, score_threshold = 0.05))    
    inference_res=[]

    for index in range(len(dataset.annotations)):
        labels_path= "../labels" #Путь к папке с предсказаниями 
        img_name = dataset.images[index].split('/')[-1].split('.')[0]
        if (os.path.exists(os.path.join(labels_path,img_name+'.txt'))):
            predictions_file = open(os.path.join(labels_path,img_name+'.txt'))
            predictions = [str.strip(s) for s in predictions_file.readlines()]
            predictions_file.close()
        else:
            predictions = []

        description = LADDDataSET.parse_voc_xml(ET.parse(dataset.annotations[index]).getroot())

        size = (int(description['annotation']['size']['width']),int(description['annotation']['size']['height']))
        boxes = []
        scores = []
        outputs={}
        for p in [convert_yolo_to_pixels(size,p) for p in predictions]:
            boxes.append([p[0][0],p[0][1],p[1][0],p[1][1]])
            scores.append(p[2])

        outputs["boxes"] = torch.as_tensor(boxes, dtype=torch.float32)
        outputs["scores"] = torch.as_tensor(scores, dtype=torch.float32)
        outputs["labels"] = torch.ones((len(predictions),), dtype=torch.int64)

        boxes = []

        for l in description['annotation']['object']:
            bb = l['bndbox']
            boxes.append([int(bb['xmin']), int(bb['ymin']), int(bb['xmax']), int(bb['ymax'])])

        target = {}
        target["boxes"] = torch.as_tensor(boxes, dtype=torch.float32)
        target["labels"] = torch.ones((len(description['annotation']['object']),), dtype=torch.int64)

        res = [target], [outputs]
        inference_res.append(res)
    print('Yolo5')
    print(evaluate_res(inference_res, iou_threshold = 0.5, score_threshold = 0.05))    

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Inference for ../../ladd-and-weights/dataset/LADD/summer_nnovgorod_2021, computing mAp : 
(0.26882707262684113, 0.17857142857142855)
Yolo5
(0.47312974920078166, 0.3125)
Inference for ../../ladd-and-weights/dataset/LADD/spring_korolev_2019, computing mAp : 
(0.7382039705656762, 0.39102564102564097)
Yolo5
(0.643101950253254, 0.42194092827004226)
Inference for ../../ladd-and-weights/dataset/LADD/summer_moscow_2019, computing mAp : 
(0.7921849366180698, 0.42948717948717946)
Yolo5
(0.8252919301107633, 0.6666666666666667)
Inference for ../../ladd-and-weights/dataset/LADD/summer_tambov_2019, computing mAp : 
(0.857478241340623, 0.4875346260387812)
Yolo5
(0.9344204939972625, 0.7860262008733624)
Inference for ../../ladd-and-weights/dataset/LADD/winter_moscow_2018, computing mAp : 
(0.9566707572592454, 0.5851528384279475)
Yolo5
(0.9486993499313932, 0.7478753541076487)
Inference for ../../ladd-and-weights/dataset/full_train_ds, computing mAp : 
(0.820090417591393, 0.4446221511395441)
Yolo5
(0.864

In [14]:
# !python ./yolov5/detect.py --augment --weights ./yolov5/runs/train/exp3/weights/best.pt --source ../../../git/ladd-and-weights/dataset/full_train_ds/JPEGImages --imgsz 1984 --conf-thres 0.05 --iou-thres 0.01 --project predict --nosave --save-txt --save-conf

In [15]:
# !mv ./predict/exp/labels ../